In [ ]:
# 20 books per 13 genres per year --> years back to 2011
# for one year: 280 book covers
# for five years: 1400 book covers

#links on links on links

#fiction, historical fiction, mystery and thriller, romance, ROMANTASY (AXED), fantasy, science fiction, horror,
#young adult fantasy and science fiction, young adult fiction, nonfiction, memoir and autobiography, history and autobiography humor

#NEW LIST: fiction, historical fiction

##FUTURE REFERENCE: TRICKY ANALYSIS DECISION: NOT ALL THE GENRES WE INITIALLY CHOSE WERE FOUND IN ALL THE YEARS- WE HAD TO GO BACK AND
#ALTER OUR GENRE LIST TO REMAIN CONSISTENT THROUGH THE YEARS --> ROMANTASY GONE

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

In [ ]:

# Define image parameters
img_height, img_width = 224, 224
batch_size = 32

# Load the CSV files
train_df = pd.read_csv('/content/train_book_covers.csv')
val_df = pd.read_csv('/content/test_book_covers.csv')

# Encode the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df['label_column'])  # Replace 'label_column' with actual column name
val_labels = label_encoder.transform(val_df['label_column'])

# Convert labels to categorical if needed
train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)

# Image preprocessing functions
def process_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])
    img = img / 255.0  # Normalize
    return img, label

# Create TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_df['image_path_column'], train_labels))  # Replace 'image_path_column'
train_ds = train_ds.map(process_image).batch(batch_size).shuffle(buffer_size=1000)

val_ds = tf.data.Dataset.from_tensor_slices((val_df['image_path_column'], val_labels))  # Replace 'image_path_column'
val_ds = val_ds.map(process_image).batch(batch_size)

# Data augmentation can be added using tf.image functions, e.g.,:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)
    # Add other augmentations as needed
    return image, label

train_ds = train_ds.map(augment)


KeyError: 'label_column'

In [ ]:
# Load pre-trained ResNet50 with ImageNet weights, excluding the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the ResNet50 layers

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer for genres

# Build the full model
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
epochs = 10  # Set the number of epochs

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)


In [ ]:
# Unfreeze specific layers for fine-tuning --> optional
for layer in base_model.layers[-10:]:  # Unfreeze the last 10 layers, for example
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_epochs = 5
history_fine = model.fit(
    train_generator,
    epochs=fine_tune_epochs,
    validation_data=val_generator
)


In [ ]:
test_loss, test_accuracy = model.evaluate(val_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")
